In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import dlib

Using TensorFlow backend.


In [2]:

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [3]:
#you can download the pretrained weights from the following link 
#https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
#or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [4]:
def preprocess_image(image_path):
    #img = load_img(image_path, target_size=(224, 224))
    #img = img_to_array(img)
    #img = np.expand_dims(image_path, axis=0)
    img = preprocess_input(image_path)
    return img

In [5]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [6]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [10]:
import cv2
import sys
import glob
from time import localtime, strftime
import os
import time

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

video_capture.set(3, 640) # set video width 
video_capture.set(4, 480) # set video height 
threshold=0.2
(width, height) = (224, 224)
idx=0
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 800,600)


a=[]
ph=[]
b={}
images=glob.glob('dataset/*.jpg')
for i in range(len(images)):
    image = cv2.imread(images[i])
    face_resize = cv2.resize(image, (224, 224))
    face_resize = face_resize.reshape(1,224,224,3)
    img1_representation = vgg_face_descriptor.predict(preprocess_image(face_resize))[0,:]
    b = {i:img1_representation}
    a.append(b)
    ph.append(os.path.basename(images[i]))
print(len(a))

while True:
    start_time = time.time()
    processing_start_time = time.time()
    time_counter = 0
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        #images=glob.glob('dataset/*.jpg')
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height))
        face_resize = face_resize.reshape(1,224,224,3)
        img1_representation = vgg_face_descriptor.predict(preprocess_image(face_resize))[0,:]
        
        cosines=[]
        for i in range(len(a)):
            cosine_similarity = findCosineSimilarity(img1_representation, a[i][i])
            cosines.append(cosine_similarity)
        
        minimum = cosines.index(min(cosines))
        #percent = (0.4-float(cosines[minimum]))*100/0.4
        #print(str(percent)+'%')
        print(cosines[minimum])
        print(images[minimum])
        print('-----------------------------')
        
        if float(cosines[minimum])<0.22:
            text=str(ph[minimum])
            
        if float(cosines[minimum])>0.25:
            text='not found'
            idx=len(a)
            #face_name = input('\n enter Directory name end press ==> ')
            write_name = 'dataset/face.'+str(idx)+'.jpg'
            base_name = 'face.'+str(idx)+'.jpg'
            cv2.imwrite(write_name, face)
            images.append(write_name)
            ph.append(base_name)
            b={idx:img1_representation}
            a.append(b)
            print('Person added')
            print('---------------------')
        
        cv2.putText(rrec,text,(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)        
        
    cv2.imshow('image', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

2
0.08292549848556519
dataset/face.1.jpg
-----------------------------
0.04371178150177002
dataset/face.1.jpg
-----------------------------
0.19212859869003296
dataset/0004_01.jpg
-----------------------------
0.08876395225524902
dataset/face.1.jpg
-----------------------------
0.052476704120635986
dataset/face.1.jpg
-----------------------------
0.18071526288986206
dataset/0004_01.jpg
-----------------------------
0.08551609516143799
dataset/face.1.jpg
-----------------------------
0.05181467533111572
dataset/face.1.jpg
-----------------------------
0.20288580656051636
dataset/0004_01.jpg
-----------------------------
0.10538369417190552
dataset/face.1.jpg
-----------------------------
0.07235085964202881
dataset/face.1.jpg
-----------------------------
0.13891267776489258
dataset/0004_01.jpg
-----------------------------
0.10935831069946289
dataset/face.1.jpg
-----------------------------
0.07885009050369263
dataset/face.1.jpg
-----------------------------
0.15610480308532715
dataset